In [ ]:
# [Cell 1] 라이브러리 임포트 및 환경 설정
import os
import pandas as pd
import numpy as np
import joblib
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn 관련
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 한글 폰트 설정 (Colab 환경)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['axes.unicode_minus'] = False # 마이너스 깨짐 방지

# 디렉토리 구조 설정
BASE_DIR = '/content/U-Sto_AI'
DATA_PATH = f"{BASE_DIR}/dataset/data_ml/phase4_training_data.csv"
MODEL_DIR = f"{BASE_DIR}/saved_models/random_forest"
RESULT_DIR = f"{BASE_DIR}/results/plots"

for d in [MODEL_DIR, RESULT_DIR]:
    os.makedirs(d, exist_ok=True)

print("✅ 환경 설정 및 디렉토리 생성 완료!")

In [ ]:
# [Cell 2] 데이터 로드 및 분할
# Phase 4에서 이미 정해둔 '데이터세트구분' 컬럼을 활용합니다.

print("📂 데이터 로드 중...")
df = pd.read_csv(DATA_PATH)

# 입력 변수(X) 정의 (모델에 들어갈 Feature)
# Phase 4에서 '_Code'로 변환한 값들과 수치형 데이터만 사용합니다.
features = [
    '내용연수', '취득금액', '부서가혹도', '가격민감도', '장비중요도', 
    '리드타임등급', '취득월',
    'G2B목록명_Code', '물품분류명_Code', '운용부서코드_Code', '캠퍼스_Code'
]
target = '실제수명' # 예측 목표: 총 수명

# 데이터셋 분리 (Phase 4의 로직 준수)
train_mask = df['데이터세트구분'] == 'Train'
valid_mask = df['데이터세트구분'] == 'Valid'
test_mask  = df['데이터세트구분'] == 'Test'
pred_mask  = df['데이터세트구분'] == 'Prediction' # (== 학습데이터여부 'N')

X_train = df.loc[train_mask, features]
y_train = df.loc[train_mask, target]

X_val = df.loc[valid_mask, features]
y_val = df.loc[valid_mask, target]

X_test = df.loc[test_mask, features]
y_test = df.loc[test_mask, target]

# 예측 대상 (현재 운용 중인 자산)
X_pred_target = df.loc[pred_mask, features]
df_pred_final = df.loc[pred_mask].copy()

print(f"✅ 데이터 분할 완료")
print(f"   - Train (학습): {X_train.shape}")
print(f"   - Valid (검증): {X_val.shape}")
print(f"   - Pred  (실전): {X_pred_target.shape}")

In [ ]:
# [Cell 3] Phase 5-1. 베이스라인 모델링
print("\n🚀 [Phase 5-1] 베이스라인(기본) 모델 학습 시작...")

# 1. 모델 초기화 (기본값)
rf_base = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# 2. 학습 (Train 데이터만 사용)
rf_base.fit(X_train, y_train)

# 3. 검증 (Valid 데이터 사용)
y_val_pred = rf_base.predict(X_val)

# 4. 성능 평가
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"📊 [Baseline Result]")
print(f"   - RMSE (오차): {rmse:.4f} 년")
print(f"   - MAE  (오차): {mae:.4f} 년")
print(f"   - R² (설명력): {r2:.4f}")

In [ ]:
# [Cell 4] Phase 5-2. 하이퍼파라미터 튜닝
print("\n🔧 [Phase 5-2] GridSearchCV 최적화 시작 (시간이 좀 걸릴 수 있습니다)...")

# 1. 파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],  # 나무의 개수
    'max_depth': [10, 20, None],      # 나무의 깊이 (과적합 제어)
    'min_samples_split': [2, 5],      # 분할 조건
    'min_samples_leaf': [1, 2, 4]     # 잎 노드 최소 샘플
}

# 2. K-Fold 설정 (5등분)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 3. Grid Search 설정
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=kf,
    scoring='neg_root_mean_squared_error', # RMSE를 기준으로 최적화
    n_jobs=-1,
    verbose=1
)

# 4. 학습 수행
grid_search.fit(X_train, y_train)

# 5. 최적 결과 도출
best_rf = grid_search.best_estimator_
print(f"🎉 최적 파라미터: {grid_search.best_params_}")

# 6. Test 셋(최종 보류 데이터)으로 최종 검증
y_test_pred = best_rf.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
final_r2 = r2_score(y_test, y_test_pred)

print(f"🏆 [Final Test Result] RMSE: {final_rmse:.4f}, R²: {final_r2:.4f}")

In [ ]:
# [Cell 5] Phase 5-3. Feature Importance 분석 및 시각화
print("\n🔍 [Phase 5-3] 중요 변수 분석...")

# 중요도 추출
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1] # 내림차순 정렬

# 시각화
plt.figure(figsize=(10, 6))
plt.title("Feature Importances (Random Forest)", fontsize=15)
sns.barplot(x=importances[indices], y=[features[i] for i in indices], palette="viridis")
plt.xlabel("Relative Importance")
plt.tight_layout()

# 그래프 저장
plot_path = f"{RESULT_DIR}/rf_feature_importance.png"
plt.savefig(plot_path)
plt.show()

print(f"💾 중요도 그래프 저장 완료: {plot_path}")

# (옵션) 모델 경량화 제안
low_impact_features = [features[i] for i in indices if importances[i] < 0.01]
if low_impact_features:
    print(f"💡 [Tip] 영향력이 낮은 변수들: {low_impact_features}")
    print("   -> 추후 모델 경량화 시 이 변수들을 제외하고 재학습을 고려해볼 수 있습니다.")

In [ ]:
# [Cell 6] Phase 6. 최종 예측 및 결과 저장
print("\n🔮 [Phase 6] 운용 자산 수명 예측 수행...")

# 1. Prediction 데이터셋에 대해 '총 수명' 예측
pred_total_life = best_rf.predict(X_pred_target)

# 2. 결과 매핑 및 잔여수명 계산
# 공식: 예측잔여수명 = AI가_예측한_총수명 - 현재_운용연차
df_pred_final['예측수명'] = pred_total_life
df_pred_final['예측잔여수명'] = df_pred_final['예측수명'] - df_pred_final['운용연차']

# 3. 마이너스 값 보정 (이미 수명이 다한 것으로 예측된 경우 0으로 처리)
# or 분석을 위해 마이너스 값을 그대로 두고 '교체 1순위'로 활용할 수도 있음
# 여기서는 0으로 보정하되, 원본 예측값도 남깁니다.
df_pred_final['예측잔여수명_Raw'] = df_pred_final['예측잔여수명']
df_pred_final['예측잔여수명'] = df_pred_final['예측잔여수명'].clip(lower=0)

# 4. 모델 및 파라미터 저장
joblib.dump(best_rf, f"{MODEL_DIR}/best_model.pkl")
with open(f"{MODEL_DIR}/params.json", 'w') as f:
    json.dump(grid_search.best_params_, f)

# 5. 성능 리포트 CSV 저장
perf_df = pd.DataFrame({
    'Model': ['RandomForest'],
    'RMSE': [final_rmse],
    'R2': [final_r2],
    'Best_Params': [str(grid_search.best_params_)]
})
perf_df.to_csv(f"{BASE_DIR}/results/performance_comparison.csv", index=False)

# 6. 최종 결과 파일 저장 (Phase 6 산출물)
final_output_path = f"{BASE_DIR}/dataset/data_ml/phase6_prediction_result.csv"
# 주요 컬럼 위주로 정렬하여 저장
output_cols = [
    '물품고유번호', 'G2B목록명', '운용부서명', '취득일자', '내용연수', '운용연차',
    '예측수명', '예측잔여수명', '예측잔여수명_Raw', 
    '부서가혹도', '장비중요도' # 분석용 참고
]
df_pred_final[output_cols].to_csv(final_output_path, index=False, encoding='utf-8-sig')

print("-" * 50)
print(f"✅ 모델 저장 완료: {MODEL_DIR}/best_model.pkl")
print(f"✅ 예측 결과 저장 완료: {final_output_path}")
print(f"👉 예측 대상 {len(df_pred_final)}건에 대한 분석이 완료되었습니다.")